# 511. Game play analysis II.
[Problem description](https://leetcode.com/problems/game-play-analysis-ii/description/)

In [1]:
import sys
sys.version

'3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]'

In [3]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]')\
    .appName('LeetCode-512-PySpark')\
    .getOrCreate()

In [4]:
# Data 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import to_date, col
schema = StructType([
    StructField('player_id', IntegerType(), False),
    StructField('device_id', IntegerType(), False),
    StructField('date', StringType(), False),  # later parse to event_date: Date
    StructField('games_played', IntegerType(), False),
])

activity = spark.createDataFrame(
    [[1, 2, '2016-03-01', 5], [1, 2, '2016-05-02', 6], [2, 3, '2017-06-25', 1], [3, 1, '2016-03-02', 0], [3, 4, '2018-07-03', 5]],
    schema=schema)
activity = activity.withColumn('event_date', to_date(col('date'), 'yyyy-MM-dd')).drop('date')
activity.show()


+---------+---------+------------+----------+
|player_id|device_id|games_played|event_date|
+---------+---------+------------+----------+
|        1|        2|           5|2016-03-01|
|        1|        2|           6|2016-05-02|
|        2|        3|           1|2017-06-25|
|        3|        1|           0|2016-03-02|
|        3|        4|           5|2018-07-03|
+---------+---------+------------+----------+



**Write a SQL query that reports the device that is first logged in for each player.**

In [6]:
# Solution
from pyspark.sql import Window
from pyspark.sql.functions import first_value, col

window = Window.partitionBy('player_id').orderBy('event_date')
 
df = activity.withColumn('is_first_login', col('event_date') == first_value(col('event_date')).over(window))
df.filter(df.is_first_login == 'true')\
    .select(col('player_id'), col('device_id'))\
    .show()

+---------+---------+
|player_id|device_id|
+---------+---------+
|        1|        2|
|        2|        3|
|        3|        1|
+---------+---------+

